In [1]:
def Auto_pro(df):
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    print(df)
    print("CHECKING NAN")
    print(df.isna().sum())
    print(df.dtypes)
    ilist=df.columns[:len(df.columns)-1]
    for i in ilist:
        print(df[i].isna().sum())
        if df[i].isna().sum()!=0:
            print("Nans in this column:",i)
            if df[i].dtype=="object":
                df[i]=df[i].fillna(df[i].mode())
            else:
                df[i]=df[i].fillna(df[i].mean())
        else:
            print("No Nan in this column:",i)
            
    df[depen].dropna(inplace=True)
    print("--------------------------------------------------")
    print("CHECKING ERROR DETECTION")
    sns.boxplot(data=df)
    plt.show()
    from scipy.stats import norm
    for e in ilist:
        if df[e].dtypes != "object":
            plt.subplot(1,2,1)
            plt.hist(df[e],ec="y",bins=20)
            plt.subplot(1,2,2)
            sns.distplot(df[e],fit=norm)
            plt.show()
            print("Column Name :",e)
            print("Distribution :",df[e].skew())
            if df[e].skew().round()==0:
                print("It is a norma distribution=>Mean std method")
                print(df[e].describe())
                m=df[e].mean()
                s=df[e].std()
                lower=m-(3*s)
                upper=m+(3*s)
                print("lower limit :",lower)
                print("upper limit :",upper)
                if df[e][(df[e]<lower) | (df[e]>upper)].count()==0:
                    print("No Errors in this column :",e)
                else:
                    df[e]=df[e].map(lambda Z : m if Z<lower else m if Z>upper else Z)
                    print("Rechecking the errors after caping")
                    print(df[(df[e]<lower) | (df[e]>upper)])
                
            else:
                print("It is a skew distribution => IQR method")
                print(df[e].describe())
                q1=df[e].quantile(0.25)
                q3=df[e].quantile(0.75)
                IQR=q3-q1
                lower1=q1-(1.5*IQR)
                upper1=q3+(1.5*IQR)
                print("lower limit :",lower1)
                print("upper limit :",upper1)
                if df[e][(df[e]<lower1) | (df[e]>upper1)].count()==0:
                    print("No Errors in this Column :",e)
                else:
                    print("Errors are there in this Column :",e)
                    df[e]=df[e].map(lambda A : df[e].mean() if A<lower1 else df[e].mean() if A>upper1 else A )
                    print("Rechecking the errors after caping")
                    print(df[(df[e]<lower1) | (df[e]>upper1)])
        elif df[e].dtypes=="Object":
            print(df[e].value_counts().plot(kind="bar"))
    print(df)        
    print("---------------------------------------------------------")        
    depen=df.columns[-1]
    if df[depen].dtypes!="object":
        d=input("dependent column Error effected  yes or no:")
        d=d.capitalize()
        if d=="Yes":
            print("Trimming the Dependent column ")
            if df[depen].skew().round()==0:
                print("normal distribution")
                m=df[depen].mean()
                s=df[depen].std()
                lower2=m-(3*s)
                upper2=m+(3*s)
                df=df[(df[depen]>=lower2) & (df[depen]<=upper2)]
                print(df)
            else:
                print("skew distribution")
                q1=df[depen].quantile(0.25)
                q3=df[depen].quantile(0.75)
                Iqr=q3-q1
                lower3=q1-(1.5*Iqr)
                upper3=q3+(1.5*Iqr)
                df=df[(df[depen]>=lower3) & (df[depen]<=upper3)]
                print(df)
        else:
            print("Dependent column not Error effected")
    print(df) 
    df1=df
    print("-------------------------------------------------------------------")        
    print("ENCODING")
    o=input("Independent columns have ordinal categorical data yes or no:")
    o=o.capitalize()
    if o=="Yes":
        print("Use the comma in between muliple columns entered ")
        olist=input("enter the Ordinal data columns:").split(",")
        for r in olist:
            from sklearn.preprocessing import OrdinalEncoder
            orde=OrdinalEncoder()
            orde.fit(df1[[r]])
            orde.transform(df1[[r]])
            df1[r]=orde.transform(df1[[r]])
    else:
        print("no Ordinal data")
        
    if df1[depen].dtypes=="object":
        from sklearn.preprocessing import LabelEncoder
        lab=LabelEncoder()
        lab.fit(df[depen])
        lab.transform(df[depen])
        df1[depen]=lab.transform(df1[depen])
    else:
        print("Dependent column have numerical data")
    df=pd.get_dummies(df[ilist],drop_first=True)
    df[depen]=df1[depen]
    print(df)
    print("------------------------------------------")
    print("DATA SEPARATION")
    x=df.drop(depen,axis=1)
    y=df[depen]
    print("Independent data shape:",x.shape)
    print("Dependent data shape:",y.shape)
    print("---------------------------------------------------------------")
    if df[depen].dtype!="float64":
        print("Imbalance Checking")
        y.value_counts().plot(kind="bar")
        imb=input("Dependent column is imbalance => yes or no:")
        imb=imb.capitalize()
        if imb=="Yes":
            from imblearn.over_sampling import SMOTE
            smt=SMOTE()
            x,y=smt.fit_resample(x,y)
        print("IT IS A CLASSIFICATION PROBLEM")
        from sklearn.model_selection import train_test_split
        from sklearn.tree import DecisionTreeClassifier
        from sklearn.svm import SVC
        from sklearn.neighbors import KNeighborsClassifier
        from sklearn.linear_model import LogisticRegression
        from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier
        dtrc=DecisionTreeClassifier()
        svc=SVC()
        knc=KNeighborsClassifier()
        lgr=LogisticRegression()
        adc=AdaBoostClassifier()
        gdc=GradientBoostingClassifier()
        rfc=RandomForestClassifier()
        mclist=[dtrc,svc,knc,lgr,adc,gdc,rfc]

        for mc in mclist:
            cl=[]
            for n in range(51):
                x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=n)
                mc.fit(x_train,y_train)
                s=mc.score(x_test,y_test)
                cl.append(s)
            s3=cll.index(max(cl))
            print("Random state => ",s3)
            print(u,"=>",max(cl))
            
        
    else:
        print("IT IS A REGRESSION PLOBLEM")
        from sklearn.model_selection import train_test_split
        from xgboost import XGBRegressor
        from sklearn.linear_model import LinearRegression
        from sklearn.tree import DecisionTreeRegressor
        from sklearn.svm import SVR
        from sklearn.neighbors import KNeighborsRegressor
        from sklearn.ensemble import AdaBoostRegressor,RandomForestRegressor,GradientBoostingRegressor
        xgb=XGBRegressor()
        lreg=LinearRegression()
        dtrr=DecisionTreeRegressor()
        svr=SVR()
        knr=KNeighborsRegressor()
        adr=AdaBoostRegressor()
        rfr=RandomForestRegressor()
        gdr=GradientBoostingRegressor()
        mrlist=[xgb,lreg,dtrr,svr,knr,adr,rfr,gdr]
        for mr in mrlist:
            rl=[]
            for u in range(51):
                x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=u)
                mr.fit(x_train,y_train)
                s1=mr.score(x_test,y_test)
                rl.append(s1)
            s4=rl.index(max(rl))
            print("Random state => ",s4)
            print(mr,"=>",max(rl))
    
                
            
                
            